In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from numpy import cos, arctan, log, mean

In [2]:
class NN:
    
    def __init__(self):
        
        self.BatchNormalization = layers.BatchNormalization
    
    # Parameters needed for NN architecture
    def call(self,
             model_input,
             n_layers,
             layers_dim,
             activation,
             BatchNormalization,
             L1_lambda,
             L2_lambda,
             dropout_rate,
             seed):
     
        # NN start
        x = model_input
        # Fix the initialization of weights
        initializer = tf.keras.initializers.GlorotUniform(seed)
        
        for i in range(n_layers):
            x = layers.Dense(layers_dim[i],
                             activation = activation,
                             kernel_initializer = initializer,
                             kernel_regularizer = regularizers.L1L2(l1 = L1_lambda, l2 = L2_lambda)
                            )(x)
            x = layers.Dropout(rate = dropout_rate)(x)
            if BatchNormalization:
                x = self.BatchNormalization()(x)
        
        x = layers.Dense(1,
                         kernel_initializer = initializer,
                         kernel_regularizer=regularizers.L1L2(l1 = L1_lambda, l2 = L2_lambda)
                        )(x)
        
        model_output = x
        model = keras.Model(model_input, model_output)
        # NN end
        
        return model

model = NN()

In [3]:
# Change parameters below to set NN architecture
model_NN4_64 = model.call(
                    # change the shape to match input shape
                    model_input = keras.layers.Input(shape=(150, )),
                    # number of hidden layers 
                    n_layers = 4,
                    # dimensions of layers of length {n_layers}
                    layers_dim = [64, 32, 16, 8],
                    # Applied to all layers. Common activations are relu, softmax, sigmoid, tanh
                    activation = 'sigmoid',
                    # True or False for batch normalization. BatchNorm is applied after every layer
                    BatchNormalization = True,
                    # L1_regularization
                    L1_lambda = 0,
                    # L2_regularization
                    L2_lambda = 0,
                    # dropout rate
                    dropout_rate = 0,
                    # initialization seed
                    seed = 120)
model_NN4_64.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 dense (Dense)               (None, 64)                9664      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                             

In [5]:
# Test the neural network on some function
def f(x):
    return np.mean(x**2 + arctan(x**2))

x_train = np.zeros((10000, 150, ))
y_train = np.zeros((10000, 1, ))
x_test = np.zeros((2000, 150, ))
y_test = np.zeros((2000, 1, ))
for i in range(10000):
    x_train[i,:] = np.random.rand(150,)
    y_train[i,:] = f(x_train[i,:]) # + np.random.rand()*0.01
for i in range(2000):
    x_test[i,:] = np.random.rand(150,)
    y_test[i,:] = f(x_test[i,:]) # + np.random.rand()*0.01



In [6]:
model_NN4_64.compile(
    loss = keras.losses.MeanAbsoluteError(),
    #optimizer=keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.mean_absolute_error]
)

# optional early stop
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

history = model_NN4_64.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2
                    ,callbacks=[earlystop]
                   )



Epoch 1/20
250/250 [==============================] - 2s 3ms/step - loss: 0.2358 - mean_absolute_error: 0.2358 - val_loss: 0.4468 - val_mean_absolute_error: 0.4468
Epoch 2/20
250/250 [==============================] - 1s 2ms/step - loss: 0.0352 - mean_absolute_error: 0.0352 - val_loss: 0.1045 - val_mean_absolute_error: 0.1045
Epoch 3/20
250/250 [==============================] - 1s 2ms/step - loss: 0.0228 - mean_absolute_error: 0.0228 - val_loss: 0.0585 - val_mean_absolute_error: 0.0585
Epoch 4/20
250/250 [==============================] - 1s 2ms/step - loss: 0.0187 - mean_absolute_error: 0.0187 - val_loss: 0.0500 - val_mean_absolute_error: 0.0500
Epoch 5/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0164 - mean_absolute_error: 0.0164 - val_loss: 0.0341 - val_mean_absolute_error: 0.0341
Epoch 6/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0157 - mean_absolute_error: 0.0157 - val_loss: 0.0233 - val_mean_absolute_error: 0.0233
Epoch 7/20
250/2

In [7]:
test_scores = model_NN4_64.evaluate(x_test, y_test, verbose=2)
# test_scores[0] is loss, test_score[1] is metric
print(test_scores)

63/63 - 0s - loss: 0.0154 - mean_absolute_error: 0.0154 - 81ms/epoch - 1ms/step
[0.015408921055495739, 0.015408921055495739]


In [8]:
x = x_test[123,:]
# x = np.zeros((150,))
y = f(x)
x = x.reshape(1, 150,)
y_pred = model_NN4_64.predict(x)
# Compare the real value and the predicted value
print(y, y_pred)

1/1 [==============================] - 0s 140ms/step
0.647414918015905 [[0.63668275]]
